
# 🤖 Aula 6: Criando um Chatbot que Aprende Sozinho

Neste notebook, vamos construir um chatbot que aprende com as interações dos usuários.

## 📌 O que faremos:
1. **Entender como funciona o aprendizado em AI Agents**.
2. **Fundamentação matemática**: Aprendizado por Reforço e Funções de Atualização.
3. **Implementar um chatbot que ajusta respostas com base em feedback do usuário**.
4. **Armazenar conhecimento de longo prazo com embeddings**.
5. **Testar personalização e adaptação do chatbot**.

---

## 📖 Como um Chatbot Aprende?

Chatbots tradicionais não aprendem de verdade, pois apenas processam a entrada do usuário e geram respostas com base em modelos pré-treinados. No entanto, podemos implementar aprendizado utilizando:

1. **Memória e Feedback do Usuário**: Guardar interações anteriores e modificar respostas futuras.
2. **Ajuste Probabilístico**: Ajustar pesos para respostas diferentes com base na satisfação do usuário.
3. **Embeddings e Vetores Semânticos**: Criar um banco de memória que permite ao chatbot lembrar informações com base na similaridade de texto.

### Matemática do Aprendizado por Reforço

Seja um chatbot com um conjunto de respostas possíveis $ R $. Quando um usuário interage, podemos definir uma função de recompensa $ Q(s, a) $ para cada resposta $ a $ dada um estado $ s $:

- Se o usuário dá **feedback positivo**, aumentamos a probabilidade de repetir essa resposta.
- Se o usuário dá **feedback negativo**, reduzimos a probabilidade dessa resposta e buscamos alternativas melhores.

A atualização pode ser modelada como:

$$
Q(s, a) = Q(s, a) + \alpha \cdot (r - Q(s, a))
$$

onde:
- $ \alpha $ é a taxa de aprendizado.
- $ r $ é a recompensa dada pelo usuário.
- $ Q(s, a) $ é o valor atualizado da resposta no estado.



In [ ]:

# Instale a biblioteca OpenAI se ainda não estiver instalada
!pip install openai json


In [ ]:

import openai
import json

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"


In [ ]:

# Criando um histórico de interações para aprendizado contínuo
historico = []

def chatbot_aprendiz(pergunta):
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    historico.append({"role": "assistant", "content": mensagem})
    
    # Salvando histórico em um arquivo JSON
    with open("historico.json", "w") as f:
        json.dump(historico, f, indent=4)
    
    return mensagem

# Teste do chatbot
print(chatbot_aprendiz("Qual é a capital da França?"))
print(chatbot_aprendiz("E qual é a minha cor favorita?"))


In [ ]:

# Instale as bibliotecas necessárias
!pip install openai faiss-cpu numpy json

import openai
import numpy as np
import faiss
import json

# Defina sua chave de API aqui (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"

# Criar banco de memória vetorial com FAISS
dimensao = 1536  # Tamanho dos embeddings da OpenAI
indice_memoria = faiss.IndexFlatL2(dimensao)  # Banco de dados vetorial
memorias = []  # Lista de strings armazenadas
respostas_personalizadas = {}  # Dicionário para aprendizado por reforço

# Função para obter embedding de um texto
def obter_embedding(texto):
    resposta = openai.Embedding.create(input=texto, model="text-embedding-ada-002")
    return np.array(resposta["data"][0]["embedding"], dtype=np.float32)

# Função para armazenar memória
def armazenar_memoria(texto, resposta):
    vetor = obter_embedding(texto)
    memorias.append((texto, resposta))
    indice_memoria.add(np.array([vetor]))
    respostas_personalizadas[texto] = resposta

# Função para recuperar memória mais próxima
def recuperar_memoria(texto):
    if len(memorias) == 0:
        return None
    
    vetor = obter_embedding(texto)
    D, I = indice_memoria.search(np.array([vetor]), k=1)
    return memorias[I[0][0]][1] if D[0][0] < 0.5 else None  # Se a distância for pequena, retornamos

# Função de aprendizado por reforço
def atualizar_aprendizado(texto, resposta, feedback):
    if feedback == "sim":
        respostas_personalizadas[texto] = resposta
    else:
        respostas_personalizadas[texto] = "Ainda estou aprendendo. Poderia reformular a pergunta?"

# Chatbot com aprendizado baseado em feedback
def chatbot_aprendizagem(pergunta):
    memoria_relevante = recuperar_memoria(pergunta)
    if memoria_relevante:
        return memoria_relevante
    
    historico = [{"role": "system", "content": "Você é um assistente que aprende com o usuário."},
                 {"role": "user", "content": pergunta}]
    
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )["choices"][0]["message"]["content"]

    armazenar_memoria(pergunta, resposta)
    
    # Solicitar feedback do usuário
    print(f"Bot: {resposta}")
    feedback = input("Isso ajudou? (Sim/Não) ").strip().lower()
    atualizar_aprendizado(pergunta, resposta, feedback)

    return resposta

# Teste de aprendizado
print(chatbot_aprendizagem("Me fale sobre aprendizado por reforço."))
print(chatbot_aprendizagem("O que é machine learning?"))


In [ ]:

# Criando um chatbot que lembra do usuário
usuarios = {}

def chatbot_personalizado(nome, pergunta):
    if nome not in usuarios:
        usuarios[nome] = []
    
    usuarios[nome].append(pergunta)
    
    resposta = chatbot_aprendiz(pergunta)
    
    return f"{nome}, baseado no que já conversamos, aqui está minha resposta: {resposta}"

# Teste de personalização
print(chatbot_personalizado("Carlos", "Qual celular devo comprar?"))
print(chatbot_personalizado("Carlos", "Qual minha preferência de marca?"))



## 🎯 Desafio para os alunos!
1. Criar um chatbot que lembra informações do usuário ao longo do tempo.
2. Implementar um mecanismo de feedback para melhorar respostas.
3. Testar personalização e adaptação do chatbot em conversas mais longas.

---
